In [19]:
import json
import os
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import MultiLineString
from shapely.wkt import loads

In [20]:
user_name = 'baoyuliu'

# create a path to the GeoJSON file
root = os.path.join('/Users', user_name, 'Documents', 'GitHub', 'G877_Alivand')
fn = 'Roads_small.geojson'
full_path = os.path.join(root, fn)

# use geopandas to read in the GeoJSON file
roads = gpd.read_file(full_path)

# create a new column that contains the length of each road
viz_roads = roads.copy()
print(viz_roads.head())

#check the data type of the geometry column
print(viz_roads['geometry'].dtype)
print(viz_roads.type)

   fid  OBJECTID                                DKM_UUID  OBJEKTART  NAME  \
0   95      2126  {84F4C50E-E009-4DB8-AF3C-DEE20F3177F6}       1700  None   
1  113      2145  {6408AD08-C7B5-4602-81D7-A9B7FBFFD5FF}       1700  None   
2  114      2146  {578A6D84-CC7B-4687-B7C6-79313525392A}       1700  None   
3  163      2195  {BF0A1AEF-817C-45DA-A33C-6C37903D19BB}       1500  None   
4  169      2201  {C3EB4DD7-7E8F-4BB5-9DFD-BEE78567F8A7}       1400  None   

  ROUTENNUMMER   SG  STUFE  KUNSTBAUTE  BEFAHRBARKEIT  ...  OVERRIDE1 OFF2  \
0         None  200      0         100            -99  ...       None  NaN   
1         None  200      0         100            -99  ...       None  NaN   
2         None  200      0         100            -99  ...       None  NaN   
3         None  200      0         100              1  ...       None  NaN   
4         None  200      0         100              1  ...       None  NaN   

   OFF3  CUT_BEGIN_FB  CUT_END_FB  EXTREMITY_TYPE2  NAME_LANG  INVER

In [21]:
class SimpleEdgeGraph:
    def __init__(self):
        self.edges = []

    def calculate_distance(self, v, w):
        return ((w[0] - v[0]) ** 2 + (w[1] - v[1]) ** 2) ** 0.5

    def add_multilinestring(self, multilinestring):
        segments = multilinestring.replace('MULTILINESTRING ', '').strip().strip('()').split('), (')
        total_length = 0.0
        for segment in segments:
            points = segment.split(', ')
            segment_length = 0.0
            last_point = None
            for point in points:
                current_point = tuple(map(float, point.split()))
                if last_point:
                    distance = self.calculate_distance(last_point, current_point)
                    segment_length += distance
                last_point = current_point
            total_length += segment_length
            self.edges.append((segment, segment_length))
        return total_length

    def print_graph(self, limit=10):
        for index, (edge, length) in enumerate(self.edges):
            if index >= limit:
                break
            print(f"Segment {edge} has length {length:.2f}")


In [26]:
graph = SimpleEdgeGraph()

# Process each road entry in the dataset
for index, row in viz_roads.iterrows():

    geom = row['geometry']
    # Check if geometry is already a MultiLineString or needs to be loaded from WKT
    if isinstance(geom, str):
        try:
            multilinestring = loads(geom)  # Attempt to convert WKT to a shapely MultiLineString object
        except:
            print(f"Error converting WKT at index {index}")
            continue
    elif isinstance(geom, MultiLineString):
        multilinestring = geom
    else:
        print(f"Skipping unsupported geometry type at index {index}")
        continue
    
    # Use the graph method to add and calculate the total length
    total_length = graph.add_multilinestring(multilinestring.wkt)
    print(f"Total length of MULTILINESTRING at index {index}: {total_length:.2f}")

# Optionally print the graph structure with a limit
graph.print_graph()


Total length of MULTILINESTRING at index 0: 287.81
Total length of MULTILINESTRING at index 1: 207.08
Total length of MULTILINESTRING at index 2: 119.98
Total length of MULTILINESTRING at index 3: 333.27
Total length of MULTILINESTRING at index 4: 258.04
Total length of MULTILINESTRING at index 5: 665.32
Total length of MULTILINESTRING at index 6: 1089.22
Total length of MULTILINESTRING at index 7: 792.24
Total length of MULTILINESTRING at index 8: 1347.52
Total length of MULTILINESTRING at index 9: 859.98
Total length of MULTILINESTRING at index 10: 760.32
Total length of MULTILINESTRING at index 11: 1684.39
Total length of MULTILINESTRING at index 12: 1876.45
Total length of MULTILINESTRING at index 13: 2404.98
Total length of MULTILINESTRING at index 14: 1170.55
Total length of MULTILINESTRING at index 15: 163.95
Total length of MULTILINESTRING at index 16: 123.24
Total length of MULTILINESTRING at index 17: 200.33
Total length of MULTILINESTRING at index 18: 859.51
Total length of 